In [ ]:
import sys
import os
import pandas as pd

# Trick: Wir fügen den Hauptordner zum Pfad hinzu, damit er 'src' findet
sys.path.append(os.path.abspath('..'))

from db import get_engine

print("--- Start Verbindungstest ---")

try:
    # 1. Engine holen
    engine = get_engine()
    
    # 2. Verbindung öffnen
    with engine.connect() as conn:
        print("✅ VERBINDUNG ERFOLGREICH!")
        print("Datenbank antwortet.")
        
        # 3. Datenbank-Name abfragen
        df = pd.read_sql("SELECT current_database();", conn)
        display(df)

except Exception as e:
    print("❌ FEHLER:")
    print(e)

--- Start Verbindungstest ---
✅ VERBINDUNG ERFOLGREICH!
Datenbank antwortet.


,current_database
0,mimic


In [ ]:
import sys
import os
from sqlalchemy import text

# Pfad-Setup
sys.path.append(os.path.abspath('..'))
from db import get_engine

print("Erstelle Tabelle 'cohort_aki'...")

# 1. SQL-Datei einlesen
with open('../sql/create_cohort_aki.sql', 'r', encoding='utf-8') as f:
    sql_script = f.read()

# 2. Ausführen
engine = get_engine()

try:
    with engine.connect() as conn:
        # Wir führen das Skript aus
        conn.execute(text(sql_script))
        # WICHTIG: Commit, damit die Tabelle dauerhaft gespeichert wird!
        conn.commit() 
        print("✅ Tabelle 'cohort_aki' erfolgreich erstellt!")
        
except Exception as e:
    print("❌ Fehler beim Erstellen der Tabelle:")
    print(e)

Erstelle Tabelle 'cohort_aki'...
✅ Tabelle 'cohort_aki' erfolgreich erstellt!


In [ ]:
import pandas as pd
from db import get_engine, text

engine = get_engine()

# Einfacher Test: Zähle die Zeilen in unserer neuen Tabelle
query = "SELECT count(*) as anzahl, sum(intervention_dialysis) as mit_dialyse FROM cohort_aki"

with engine.connect() as conn:
    df = pd.read_sql(text(query), conn)

display(df)

,anzahl,mit_dialyse
0,11414,1330


In [ ]:
import pandas as pd
from db import get_engine, text

# 1. Daten laden
engine = get_engine()
query = """
SELECT 
    intervention_dialysis, 
    COUNT(*) as anzahl_patienten,
    SUM(outcome_death) as anzahl_verstorbene,
    ROUND(AVG(outcome_death) * 100, 2) as sterblichkeit_prozent
FROM cohort_aki
GROUP BY intervention_dialysis
ORDER BY intervention_dialysis;
"""

with engine.connect() as conn:
    df_mortality = pd.read_sql(text(query), conn)

# 2. Beschriftung für schönere Anzeige hinzufügen
df_mortality['Gruppe'] = df_mortality['intervention_dialysis'].map({
    0: 'Ohne Dialyse (Konservativ)', 
    1: 'Mit Dialyse'
})

# 3. Tabelle anzeigen
# Wir setzen die 'Gruppe' als Index, damit es sauber aussieht
display(df_mortality.set_index('Gruppe')[['anzahl_patienten', 'anzahl_verstorbene', 'sterblichkeit_prozent']])

,anzahl_patienten,anzahl_verstorbene,sterblichkeit_prozent
Gruppe,,,
Ohne Dialyse (Konservativ),10084,2063,20.46
Mit Dialyse,1330,469,35.26
